In [5]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

df = pd.read_csv("../data/area_region_mapping.csv")

geolocator = Nominatim(user_agent="aeso-load-project")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=2)  

def get_lat_lon(name):
    # Add province + country to improve accuracy
    query = f"{name}, Alberta, Canada"
    loc = geocode(query)
    if loc is None:
        return None, None
    return loc.latitude, loc.longitude

lats = []
lons = []

for name in df["location_name"]:
    lat, lon = get_lat_lon(name)
    lats.append(lat)
    lons.append(lon)
    print(f"{name}: {lat}, {lon}")

df["latitude"] = lats
df["longitude"] = lons

df.to_csv("../data/area_region_mapping_with_coords.csv", index=False)
print("Saved to area_region_mapping_with_coords.csv")

Medicine Hat: 50.04303, -110.679016
Calgary: 51.0456064, -114.057541
Lloydminster: 53.279995, -110.00885
Rainbow Lake: 58.502505, -119.400279
High Level: 58.5115361, -117.1438756
Peace River: 56.2337111, -117.2910839
Grande Prairie: 55.17108, -118.7949873
High Prairie: 55.4325044, -116.4836198
Grande Cache: 53.8888135, -119.1185658
Valleyview: 55.066667, -117.283333
Fox Creek: 54.4061853, -116.8004913
Fort McMurray: 56.7291997, -111.3885221
Swan Hills: 54.7180657, -115.4008258
Athabasca: 54.7214539, -113.2857941
Cold Lake: 54.46046, -110.192871
Hinton: 53.3990512, -117.5886772
Drayton Valley: 53.2205548, -114.9832127
Wetaskiwin: 52.968492, -113.36792
Wainwright: 52.8402725, -110.8514344
Fort Saskatchewan: 53.7128571, -113.214894
Abraham Lake: 52.2063903, -116.4525891
Red Deer: 52.2690628, -113.8141464
Alliance: 52.4335554, -111.7832364
Provost: 52.4137383, -110.639576
Caroline: 52.0943034, -114.7391684
Didsbury: 51.6567661, -114.1375682
Wabamun: 53.5592733, -114.4783142
Hanna: 51.64455

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Vauxhall, Alberta, Canada',), **{}).
Traceback (most recent call last):
  File "/home/behzad/612FinalProjectGroup3/.venv/lib/python3.12/site-packages/geopy/adapters.py", line 298, in get_text
    page = self.urlopen(req, timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 515, in open
    response = self._open(req, data)
               ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 532, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 492, in _call_chain
    result = func(*args)
             ^^^^^^^^^^^
  File "/usr/lib/python3.12/urllib/request.py", line 1392, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Vauxhall: 50.0691848, -112.1076913
Fort Macleod: 49.7267379, -113.4076001
Lethbridge: 49.6945782, -112.8331033
Glenwood: 53.5364258, -113.602552
Vegreville: 53.5, -112.05
Airdrie: 51.28597, -114.01062
Edmonton: 53.5462055, -113.491241
Saved to area_region_mapping_with_coords.csv
